In [ ]:
import os

import duckdb
import pandas as pd

In [2]:
con = duckdb.connect("binge.duckdb")

#-------------------------------------------------------------------
# 1) rating_*.txt dosyalarını tabloya yükle
#-------------------------------------------------------------------
rating_paths = [
    "C:/you_do/rating_1.txt",
    "C:/you_do/rating_1.txt",
    "C:/you_do/rating_1.txt",
    "C:/you_do/rating_1.txt"
]


for idx, path in enumerate(rating_paths):
    full_path = os.path.abspath(path)
    create_temp_table = f"""
    CREATE TABLE rating_{idx} AS
    SELECT
       CAST(column0 AS INT)      AS movie_id,
       CAST(column1 AS INT)      AS user_id,
       CAST(column2 AS VARCHAR)  AS rating_date,
       CAST(column3 AS INT)      AS rating
    FROM read_csv_auto('{full_path}',
        delim=',',
        header=false,
        ignore_errors=true,
        sample_size=-1
    );
    """
    con.execute(create_temp_table)
    print(f"{path} -> rating_{idx} tablosu oluşturuldu.")

# Tüm rating tablolarını tek tabloda toplayalım
union_query = """
CREATE TABLE IF NOT EXISTS ratings AS
SELECT * FROM rating_0
UNION ALL
SELECT * FROM rating_1
UNION ALL
SELECT * FROM rating_2
UNION ALL
SELECT * FROM rating_3
"""
con.execute(union_query)

# Geçici tabloları silelim (isteğe bağlı)
for idx in range(len(rating_paths)):
    con.execute(f"DROP TABLE rating_{idx}")

# Test: ratings tablosundaki satır sayısı
row_count = con.execute("SELECT COUNT(*) FROM ratings").fetchone()[0]
print(f"ratings tablosunda toplam satır: {row_count}")

#-------------------------------------------------------------------
# 2) movie_titles.csv dosyasını tabloya yükle
#-------------------------------------------------------------------
movies_path = os.path.abspath("movie_titles.csv")
create_movies_table = """
CREATE TABLE IF NOT EXISTS movie_titles AS
SELECT
   CAST(column0 AS INT)                              AS movie_id,
   CAST(NULLIF(column1, 'NULL') AS INT)              AS year, 
   CAST(column2 AS VARCHAR)                           AS movie_title
FROM read_csv_auto('movie_titles.csv',
    delim=',',
    header=false,
    ignore_errors=true,
    sample_size=-1
);
"""
con.execute(create_movies_table)
movies_count = con.execute("SELECT COUNT(*) FROM movie_titles").fetchone()[0]
print(f"movie_titles tablosunda toplam film sayısı: {movies_count}")

#-------------------------------------------------------------------
# 3) Örnek Sorgular (Debug/Test)
#-------------------------------------------------------------------

# a) ratings tablosundan örnek kayıtlar
sample_ratings = con.execute("SELECT * FROM ratings LIMIT 5").fetchdf()
print("\nratings tablosundan örnek satırlar:")
print(sample_ratings)

# b) movie_titles tablosundan örnek kayıtlar
sample_movies = con.execute("SELECT * FROM movie_titles LIMIT 5").fetchdf()
print("\nmovie_titles tablosundan örnek satırlar:")
print(sample_movies)

# c) Örneğin en çok rating almış ilk 5 filmi sorgulayalım
top_5_rated = con.execute("""
    SELECT 
       r.movie_id,
       COUNT(*) AS rating_count,
       AVG(r.rating) AS avg_rating,
       m.movie_title
    FROM ratings r
    JOIN movie_titles m ON r.movie_id = m.movie_id
    GROUP BY r.movie_id, m.movie_title
    ORDER BY rating_count DESC
    LIMIT 5
""").fetchdf()

print("\nEn çok rating almış ilk 5 film:")
print(top_5_rated)

#-------------------------------------------------------------------
# 4) Bağlantıyı kapat
#-------------------------------------------------------------------

C:/you_do/rating_1.txt -> rating_0 tablosu oluşturuldu.
C:/you_do/rating_1.txt -> rating_1 tablosu oluşturuldu.
C:/you_do/rating_1.txt -> rating_2 tablosu oluşturuldu.
C:/you_do/rating_1.txt -> rating_3 tablosu oluşturuldu.
ratings tablosunda toplam satır: 100480507
movie_titles tablosunda toplam film sayısı: 17433

ratings tablosundan örnek satırlar:
   movie_id  user_id rating_date  rating
0         1  1488844  2005-09-06       3
1         1   822109  2005-05-13       5
2         1   885013  2005-10-19       4
3         1    30878  2005-12-26       4
4         1   823519  2004-05-03       3

movie_titles tablosundan örnek satırlar:
   movie_id  year                   movie_title
0         1  2003               Dinosaur Planet
1         2  2004    Isle of Man TT 2004 Review
2         3  1997                     Character
3         4  1994  Paula Abdul's Get Up & Dance
4         5  2004      The Rise and Fall of ECW

En çok rating almış ilk 5 film:
   movie_id  rating_count  avg_rating

In [ ]:
query_tmp_movies = """
CREATE TABLE IF NOT EXISTS tmp_movies AS
SELECT
    column0::VARCHAR AS col0,
    column1::VARCHAR AS col1,
    column2::VARCHAR AS col2
FROM read_csv_auto('c:/you_do/movie_titles.csv',
    delim=',',
    header=false,
    ignore_errors=true,
    sample_size=-1,
    columns={'column0':'VARCHAR','column1':'VARCHAR','column2':'VARCHAR'}
);
"""

query_create_movie_titles = """
CREATE TABLE IF NOT EXISTS movie_titles AS
SELECT
    CASE WHEN col0='NULL' OR col0='' THEN NULL ELSE CAST(col0 AS INT) END AS movie_id,
    CASE WHEN col1='NULL' OR col1='' THEN NULL ELSE CAST(col1 AS INT) END AS year,
    col2 AS movie_title
FROM tmp_movies;
"""
#.
query_drop_tmp_movies = "DROP TABLE tmp_movies;"

con.execute(query_tmp_movies)
con.execute(query_create_movie_titles)
con.execute(query_drop_tmp_movies)

In [4]:
con = duckdb.connect(database=':memory:')  


rating_path = "rating_1.txt"
full_path = os.path.abspath(rating_path)

create_table_sql = f"""
CREATE TABLE ratings AS
SELECT
  CAST(column0 AS INT) AS movie_id,
  CAST(column1 AS INT) AS user_id,
  CAST(column2 AS VARCHAR) AS rating_date,
  CAST(column3 AS INT) AS rating
FROM read_csv_auto('{full_path}', delim=',', header=false);
"""

con.execute(create_table_sql)

movie_titles_path = "movie_titles.csv"
full_movies = os.path.abspath(movie_titles_path)

con.execute(f"""
CREATE TABLE movie_titles AS
SELECT
    CAST(NULLIF(column0, 'NULL') AS INT)  AS movie_id,
    CAST(NULLIF(column1, 'NULL') AS INT)  AS year,
    NULLIF(column2, 'NULL')::VARCHAR      AS movie_title
FROM read_csv_auto('{full_movies}',
    delim=',',
    header=false,
    ignore_errors=true,
    sample_size=-1,
    columns={{'column0':'VARCHAR','column1':'VARCHAR','column2':'VARCHAR'}}
);
""")


query_cold_start = """
    WITH movie_stats AS (
        SELECT
            movie_id,
            AVG(rating) AS avg_rating,
            COUNT(*) AS num_ratings
        FROM ratings
        GROUP BY movie_id
        HAVING COUNT(*) > 10
    )
    SELECT 
        m.movie_id,
        m.movie_title,
        m.year,
        ms.avg_rating,
        ms.num_ratings
    FROM movie_stats ms
    JOIN movie_titles m ON m.movie_id = ms.movie_id
    ORDER BY ms.avg_rating DESC
    LIMIT 10
"""

df_cold_start = con.execute(query_cold_start).df()
print("Cold Start (popüler filmler) önerisi:")
print(df_cold_start)

Cold Start (popüler filmler) önerisi:
   movie_id                                        movie_title  year  \
0      3456                                     Lost: Season 1  2004   
1      3033   Ghost in the Shell: Stand Alone Complex: 2nd Gig  2005   
2      2102                             The Simpsons: Season 6  1994   
3      4238                                          Inu-Yasha  2000   
4        13  Lord of the Rings: The Return of the King: Ext...  2003   
5      3444              Family Guy: Freakin' Sweet Collection  2004   
6      4427                            The West Wing: Season 3  2001   
7      1476                           Six Feet Under: Season 4  2004   
8      2019                                   Samurai Champloo  2004   
9      1418  Inu-Yasha: The Movie 3: Swords of an Honorable...  2002   

   avg_rating  num_ratings  
0    4.670989         7249  
1    4.586364          220  
2    4.581296         8426  
3    4.554434         1883  
4    4.552000          1

In [5]:
df_ratings = con.execute("SELECT user_id, movie_id, rating FROM ratings").df()
print(df_ratings.head())
print("Total rating rows:", len(df_ratings))

   user_id  movie_id  rating
0  1488844         1       3
1   822109         1       5
2   885013         1       4
3    30878         1       4
4   823519         1       3
Total rating rows: 24053764


In [6]:
df_ratings['rating'] = pd.to_numeric(df_ratings['rating'], errors='coerce')


df_ratings.dropna(subset=['rating'], inplace=True)


df_ratings['rating'] = df_ratings['rating'].astype(int)


df_ratings['user_id'] = pd.to_numeric(df_ratings['user_id'], errors='coerce')
df_ratings['movie_id'] = pd.to_numeric(df_ratings['movie_id'], errors='coerce')


df_ratings.dropna(subset=['user_id','movie_id'], inplace=True)

print(df_ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24053764 entries, 0 to 24053763
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int32
 1   movie_id  int32
 2   rating    int32
dtypes: int32(3)
memory usage: 275.3 MB
None


In [7]:
con = duckdb.connect(database="binge.duckdb")

query_user_stats = """
WITH user_stats AS (
    SELECT user_id, 
           COUNT(*) AS total_ratings,
           AVG(rating) AS avg_rating,
           STDDEV(rating) AS std_rating
    FROM ratings
    GROUP BY user_id
)
SELECT user_id 
FROM user_stats
WHERE total_ratings > 10 -- Çok az puan verenleri hariç tut
AND (avg_rating + 2 * std_rating < 2 OR avg_rating - 2 * std_rating > 4);
"""


extreme_users_df = con.execute(query_user_stats).fetchdf()


query_filtered_ratings = """
CREATE TABLE IF NOT EXISTS filtered_ratings AS
SELECT * FROM ratings
WHERE user_id NOT IN (
    SELECT user_id FROM (
        WITH user_stats AS (
            SELECT user_id, 
                   COUNT(*) AS total_ratings,
                   AVG(rating) AS avg_rating,
                   STDDEV(rating) AS std_rating
            FROM ratings
            GROUP BY user_id
        )
        SELECT user_id 
        FROM user_stats
        WHERE total_ratings > 10 
        AND (avg_rating + 2 * std_rating < 2 OR avg_rating - 2 * std_rating > 4)
    ) AS extreme_users
);
"""

con.execute(query_filtered_ratings)


num_extreme_users = len(extreme_users_df)
total_users = con.execute("SELECT COUNT(DISTINCT user_id) FROM ratings").fetchone()[0]
filtered_users = con.execute("SELECT COUNT(DISTINCT user_id) FROM filtered_ratings").fetchone()[0]


{
    "Toplam Kullanıcı Sayısı": total_users,
    "Filtrelenen Uç Kullanıcı Sayısı": num_extreme_users,
    "Filtrelenmiş Kullanıcı Sayısı": filtered_users
}

{'Toplam Kullanıcı Sayısı': 480189,
 'Filtrelenen Uç Kullanıcı Sayısı': 2395,
 'Filtrelenmiş Kullanıcı Sayısı': 477794}

In [ ]:
# DuckDB bağlantısını tekrar aç
con = duckdb.connect(database="binge.duckdb")

# Kullanıcı benzerliklerini hesaplayan SQL sorgusunu çalıştır
query_user_similarities = """
WITH user_similarities AS (
    SELECT a.user_id AS user1, 
           b.user_id AS user2, 
           COUNT(*) AS common_movies,
           AVG(ABS(a.rating - b.rating)) AS avg_rating_diff
    FROM filtered_ratings a
    JOIN filtered_ratings b 
    ON a.movie_id = b.movie_id AND a.user_id != b.user_id
    GROUP BY user1, user2
    HAVING common_movies > 5  -- En az 5 ortak film izleyen kullanıcıları al
)
SELECT * FROM user_similarities
ORDER BY avg_rating_diff ASC
LIMIT 10;
"""

# Sorguyu çalıştır ve sonucu bir DataFrame olarak al
user_similarities_df = con.execute(query_user_similarities).fetchdf()

# Bağlantıyı kapat
con.close()